In [138]:
import re
import json
from bs4 import BeautifulSoup
from bs4 import NavigableString
import requests
from time import sleep
import webbrowser

In [144]:
try:
    with open("projectLinks_ban.txt", "r") as f:
        projectLinks_ban = [i for i in f.read().split('\n')]
        f.close()
except Exception:
    print(Exception)
    projectLinks_ban = []
    
try:
    with open("projectLinks_thue.txt", "r") as f:
        projectLinks_thue = [i for i in f.read().split('\n')]
        f.close()
except Exception:
    print(Exception)
    projectLinks_thue = []

def getCode( index:int, firstString:str, length:int = 3)->str:
    if len(str(index))<length:
        return firstString+'0'*(length-len(str(index)))+str(index)
    return firstString+str(index)

def getOverview(elements):
    string = ""
    current = -1
    for index, value in enumerate(elements):
        if value.name == "p":
            if value.text != "":
                current = index
                string += "{}\n".format(value.text.replace("'","").replace('"', ''))
        if index-current>0 and current!=-1:
            break
    return string

def getText(elements):
    string = ""
    for index, value in enumerate(elements):
        if value.name == "p" or value.name == "h3" or value.name == "h2":
            if value.text != "":
                string += "{}\n".format(value.text.replace("'","").replace('"', ''))
        elif value.name =="ul":
            for v in value:
                if type(v) != NavigableString:
                    string += "{}\n".format(v.text.replace("'","").replace('"', ''))
    return string, "" if not elements.find('img') else elements.find('img').get('data-src')

listdata_project = {}

projectLinks = projectLinks_thue + projectLinks_ban
new_projectLinks = list(dict.fromkeys(projectLinks))

print(len(new_projectLinks))

for index, value in enumerate(new_projectLinks):
    projectId = getCode(index+1, "Project", 3)
    print(index, value, projectId)
    listdata_project[projectId] = {}
    response = requests.get(value)
    content = BeautifulSoup(response.content, 'html.parser')
    title = content.find('div', {'class', 'project-title'})
    listdata_project[projectId]["title"] = title.text
    name = title.find('h1', {'class', 'fs-3'}).text
    listdata_project[projectId]["name"] = name
    address = title.find('div', {'class', 'mb-3 fs-8'}).text
    listdata_project[projectId]["address"] = address
    detail_content = content.find('div', {'id':'tong-quan'}).find('div', {'class', 'detail-content'})
    overview = getOverview(detail_content)
    listdata_project[projectId]["overview"] = overview
    img = detail_content.find('img', {'class',"img-overview"}).get('data-src')
    listdata_project[projectId]["img"] = img
    introduce = content.find('div', {'class', 'article-caption text-bold mb-3'}).text
    listdata_project[projectId]["introduce"] = introduce
    for value in detail_content.find('div', {'class', 'overview'}).find_all('div', {'class', 'col-6'}):
        listdata_project[projectId][value.find('div', {'class', 'title'}).text] = value.find('div', {'class', 'description'}).text
    location, locationImage = getText(content.find('div', {'id':'vi-tri'}).find('div', {'class', 'detail-content'}))
    masterPlan, masterPlanImage = getText(content.find('div', {'id':'mat-bang'}).find('div', {'class', 'detail-content'}))
    utilities, utilitiesImage = getText(content.find('div', {'id':'tien-ich'}).find('div', {'class', 'detail-content'}))
    listdata_project[projectId]["location"] = location
    listdata_project[projectId]["locationImage"] = locationImage
    listdata_project[projectId]["masterPlan"] = masterPlan
    listdata_project[projectId]["masterPlanImage"] = masterPlanImage
    listdata_project[projectId]["utilities"] = utilities
    listdata_project[projectId]["utilitiesImage"] = utilitiesImage
    listdata_project[projectId]["investorName"] = content.find('a', {'class', 'card-title link-primary fs-5 text-uppercase text-bold'}).text

with open('projectData.json', 'w', encoding='utf-8') as f:
    json.dump(listdata_project, f, ensure_ascii=False, indent=4)
    f.close()

172
0 https://houseviet.vn/khu-do-thi-moi/vinhomes-grand-park-quan-9 Project001
1 https://houseviet.vn/khu-do-thi-moi/khu-dan-cu-phong-phu-binh-chanh Project002
2 https://houseviet.vn/khu-phuc-hop/saigon-centre Project003
3 https://houseviet.vn/can-ho-chung-cu/the-6nature-da-nang Project004
4 https://houseviet.vn/khu-do-thi-moi/vinhomes-skylake Project005
5 https://houseviet.vn/can-ho-chung-cu/can-ho-masteri-millennium Project006
6 https://houseviet.vn/can-ho-chung-cu/sky-garden-3 Project007
7 https://houseviet.vn/can-ho-chung-cu/can-ho-the-tresor-quan-4 Project008
8 https://houseviet.vn/khu-phuc-hop/empire-city Project009
9 https://houseviet.vn/khu-phuc-hop/vinhomes-golden-river Project010
10 https://houseviet.vn/dat-nen-du-an/central-golden-land Project011
11 https://houseviet.vn/can-ho-chung-cu/can-ho-the-gold-view Project012
12 https://houseviet.vn/can-ho-chung-cu/can-ho-rivergate-quan-4 Project013
13 https://houseviet.vn/khu-phuc-hop/vinhomes-central-park Project014
14 https://hou

111 https://houseviet.vn/khu-phuc-hop/the-fusion Project112
112 https://houseviet.vn/can-ho-chung-cu/new-galaxy-di-an Project113
113 https://houseviet.vn/dat-nen-du-an/tien-loc-garden-dong-nai Project114
114 https://houseviet.vn/can-ho-chung-cu/geleximco-southern-star Project115
115 https://houseviet.vn/biet-thu-nha-lien-ke/du-an-van-xuan-compound-hue Project116
116 https://houseviet.vn/khu-do-thi-moi/stella-mega-city-binh-thuy-can-tho Project117
117 https://houseviet.vn/khu-do-thi-moi/nam-co-dam-bim-son-thanh-hoa Project118
118 https://houseviet.vn/khu-phuc-hop/khu-dan-cu-sonadezi-chau-duc Project119
119 https://houseviet.vn/dat-nen-du-an/the-gems-paradise-2 Project120
120 https://houseviet.vn/can-ho-chung-cu/diyas-sky Project121
121 https://houseviet.vn/biet-thu-nha-lien-ke/du-an-phuong-nam-river-park-ben-tre Project122
122 https://houseviet.vn/can-ho-chung-cu/sun-grand-marina-town Project123
123 https://houseviet.vn/can-ho-chung-cu/du-an-masterise-ba-son Project124
124 https://house